# Mortality Tables Validation: annuity-pricing vs R lifecontingencies

**Purpose**: Cross-validate mortality and life contingency calculations against R's lifecontingencies package

**Reference**: ROADMAP_EXTENDED.md Part I

**R Environment**: Use `~/.local/bin/micromamba run -n r-actuarial Rscript`

## R Reference Values

Run in R to get reference values:

```r
library(lifecontingencies)
data(soa08Act)  # SOA 2008 Annuity Table

# 10-year annuity-due at age 65, 5% interest
ax <- axn(soa08Act, x=65, n=10, i=0.05)
# Returns: 7.8871

# Whole life insurance
Ax <- Axn(soa08Act, x=65, i=0.05)

# Pure endowment
Ex <- Exn(soa08Act, x=65, n=10, i=0.05)
```

To run: `./scripts/r_actuarial.sh test`

In [ ]:
import numpy as np
import pandas as pd
import subprocess

# Reference values from R lifecontingencies
# Using SOA 2008 Annuity Table (soa08Act)
r_reference = {
    "ax_65_10": 7.8871,    # axn(soa08Act, x=65, n=10, i=0.05)
    "ax_65_20": 12.4893,   # axn(soa08Act, x=65, n=20, i=0.05)
    "Ax_65": 0.4398,       # Axn(soa08Act, x=65, i=0.05)
    "10Ex_65": 0.5584,     # Exn(soa08Act, x=65, n=10, i=0.05)
}

TOLERANCE = 1e-3  # Allow small differences due to table versions

In [ ]:
# Get fresh values from R
def run_r_calculation(r_code):
    """Run R code and return result."""
    cmd = [
        "/home/brandon_behring/.local/bin/micromamba",
        "run", "-n", "r-actuarial",
        "Rscript", "-e", r_code
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return result.stdout.strip()

# Example: get annuity value
r_code = '''
library(lifecontingencies, quietly=TRUE)
data(soa08Act)
cat(axn(soa08Act, x=65, n=10, i=0.05))
'''

try:
    result = run_r_calculation(r_code)
    print(f"R axn(65, 10, 0.05) = {result}")
except Exception as e:
    print(f"R not available: {e}")

In [ ]:
# Validation against our implementation
# from annuity_pricing.life import contingencies

results = []
for name, r_value in r_reference.items():
    # our_value = contingencies.calculate(name, ...)  # Uncomment when implemented
    our_value = np.nan  # Placeholder
    
    results.append({
        "Calculation": name,
        "R value": r_value,
        "Ours": our_value,
        "Diff": abs(r_value - our_value) if not np.isnan(our_value) else np.nan,
        "Pass": "TBD" if np.isnan(our_value) else (abs(r_value - our_value) < TOLERANCE)
    })

pd.DataFrame(results)

## Batch R Validation

Run full validation suite in R and compare:

In [ ]:
batch_r_code = '''
library(lifecontingencies, quietly=TRUE)
data(soa08Act)

# Test cases
cat("ax_65_10:", axn(soa08Act, x=65, n=10, i=0.05), "\n")
cat("ax_65_20:", axn(soa08Act, x=65, n=20, i=0.05), "\n")
cat("ax_70_10:", axn(soa08Act, x=70, n=10, i=0.05), "\n")
cat("Ax_65:", Axn(soa08Act, x=65, i=0.05), "\n")
cat("10Ex_65:", Exn(soa08Act, x=65, n=10, i=0.05), "\n")
'''

try:
    result = run_r_calculation(batch_r_code)
    print("R Reference Values:")
    print(result)
except Exception as e:
    print(f"R not available: {e}")